In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import natsort
import pandas as pd
import scipy
import math
from sklearn.model_selection import train_test_split
import librosa
import librosa.display
from numpy.ma.core import argsort

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
labelDir = '/content/drive/MyDrive/Data/ff1010bird_metadata.csv'


In [ ]:
df = pd.read_csv(labelDir,header=None,skiprows=1)
data = df.values
print(df.head)

<bound method NDFrame.head of            0  1
0      64486  0
1       2525  0
2      44981  0
3     101323  0
4     165746  0
...      ... ..
7685  168059  0
7686  164922  0
7687   80789  1
7688  104733  1
7689   40565  0

[7690 rows x 2 columns]>


In [ ]:
sortedArr = data[data[:,0].argsort()]
# sortedArr
sortedLabel=sortedArr[:,1]
sortedFileName=sortedArr[:,0]

print(sortedFileName[0:50])
print(sortedLabel[0:50])

only50Label=sortedLabel[:200]
print(only50Label.shape)

[  55   87   99  100  104  156  345  347  349  369  370  371  376  377
  378  518 1042 1043 1044 1045 1046 1050 1051 1052 1053 1054 1055 1056
 1748 1931 1932 1933 1934 1945 2151 2155 2157 2421 2432 2435 2519 2520
 2521 2522 2523 2524 2525 2526 2527 2528]
[0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 1 1 1 0 1 1 0 0 0 1 0 0 0 0 0 1
 0 0 0 0 0 0 0 0 0 0 0 0 0]
(200,)


In [ ]:
wavDir = '/content/drive/MyDrive/Data/sample200/'
listOfWavs=os.listdir(wavDir)
print(len(listOfWavs))

200


In [ ]:
sortWavDir=natsort.natsorted(listOfWavs,reverse=False)
print(sortWavDir)


['104.wav', '347.wav', '1042.wav', '1050.wav', '1052.wav', '1054.wav', '1945.wav', '2151.wav', '2525.wav', '2531.wav', '2537.wav', '3182.wav', '3187.wav', '3188.wav', '3191.wav', '5204.wav', '5929.wav', '5996.wav', '5997.wav', '7151.wav', '7173.wav', '7320.wav', '7321.wav', '7542.wav', '7543.wav', '7875.wav', '7885.wav', '7887.wav', '7991.wav', '7994.wav', '7996.wav', '9219.wav', '9221.wav', '9225.wav', '9236.wav', '11685.wav', '11692.wav', '11696.wav', '11702.wav', '11732.wav', '12699.wav', '12704.wav', '12710.wav', '12728.wav', '13735.wav', '14314.wav', '14527.wav', '14853.wav', '14909.wav', '15225.wav', '15226.wav', '15290.wav', '15426.wav', '15459.wav', '15466.wav', '15468.wav', '15470.wav', '15552.wav', '15553.wav', '15598.wav', '15657.wav', '15792.wav', '16633.wav', '16765.wav', '16837.wav', '16990.wav', '16993.wav', '16995.wav', '17004.wav', '17005.wav', '17007.wav', '17008.wav', '17054.wav', '17055.wav', '17075.wav', '17082.wav', '17136.wav', '17138.wav', '17139.wav', '17143.wa

In [ ]:
wavData = []
for file in sortWavDir:
    print(file)
    filePath=wavDir+file
    print(filePath)
    audio, sr = librosa.load(filePath)
    print('duration in seconds = ',len(audio)/sr)
    fourSecSample = audio[0*sr:10*sr]
    print('duration in seconds = ',len(fourSecSample)/sr)
    wavData.append(fourSecSample)

wavData = np.asarray(wavData)
    


104.wav
/content/drive/MyDrive/Data/sample200/104.wav
duration in seconds =  10.0
duration in seconds =  10.0
347.wav
/content/drive/MyDrive/Data/sample200/347.wav
duration in seconds =  10.0
duration in seconds =  10.0
1042.wav
/content/drive/MyDrive/Data/sample200/1042.wav
duration in seconds =  10.0
duration in seconds =  10.0
1050.wav
/content/drive/MyDrive/Data/sample200/1050.wav
duration in seconds =  10.0
duration in seconds =  10.0
1052.wav
/content/drive/MyDrive/Data/sample200/1052.wav
duration in seconds =  10.0
duration in seconds =  10.0
1054.wav
/content/drive/MyDrive/Data/sample200/1054.wav
duration in seconds =  10.0
duration in seconds =  10.0
1945.wav
/content/drive/MyDrive/Data/sample200/1945.wav
duration in seconds =  10.0
duration in seconds =  10.0
2151.wav
/content/drive/MyDrive/Data/sample200/2151.wav
duration in seconds =  10.0
duration in seconds =  10.0
2525.wav
/content/drive/MyDrive/Data/sample200/2525.wav
duration in seconds =  10.0
duration in seconds =  1

In [ ]:
print('shape===',wavData.shape)

shape=== (200, 220500)


In [ ]:
allFeature1 = []
for frame in range(len(wavData)):
  OneSample = wavData[frame]
  # Energy
  energy=np.sum(np.square(OneSample)).reshape(-1,1)
  # Spectral Centroid
  spectral_centroid = librosa.feature.spectral_centroid(y=OneSample, sr=sr,n_fft=20, hop_length=len(OneSample)+1, center = False, win_length = 15 )
  #RMS
  rms = librosa.feature.rms(y=OneSample, hop_length=len(OneSample)+1)

  featuresOneFrame=np.concatenate((energy, spectral_centroid, rms),axis = 0)
  allFeature1.append(featuresOneFrame)





In [ ]:
from librosa.feature.spectral import spectral_bandwidth
allFeature2 = []
for frame in range(len(wavData)):
  OneSample2 = wavData[frame]
  ##ZCR
  zcr=librosa.feature.zero_crossing_rate(OneSample,hop_length=len(OneSample2)+1)
  # Spectral Roll_off
  spectral_rolloff = librosa.feature.spectral_rolloff(y=OneSample2, sr=sr,n_fft=20, hop_length=len(OneSample)+1, center = False, win_length = 15 )
  #Spectral Bandwidth
  spectral_bandwidth = librosa.feature.spectral_bandwidth(y=OneSample2, hop_length=len(OneSample)+1)

  featuresOneFrame2=np.concatenate((energy, spectral_rolloff, spectral_centroid),axis = 0)
  allFeature2.append(featuresOneFrame2)

In [ ]:
print(energy.shape)
print(spectral_centroid.shape)
print(rms.shape)
print(featuresOneFrame.shape)

allFeature1=np.asarray(allFeature1)

print(allFeature1.shape)
allFeature1=np.reshape(allFeature1, (allFeature1.shape[0],allFeature1.shape[1]))
print(allFeature1.shape)

(1, 1)
(1, 1)
(1, 1)
(3, 1)
(200, 3, 1)
(200, 3)


In [ ]:
allFeature2=np.asarray(allFeature2)

print(allFeature2.shape)
allFeature2=np.reshape(allFeature2, (allFeature2.shape[0],allFeature2.shape[1]))
print(allFeature2.shape)

(200, 3, 1)
(200, 3)


In [ ]:
from sklearn import preprocessing
allFeature1_standardized = preprocessing.scale(allFeature1)

print(np.mean(allFeature1_standardized,axis=0))
print(np.std(allFeature1_standardized,axis=0))

[0.00000000e+00 1.25455202e-16 6.32827124e-17]
[1. 1. 1.]


In [ ]:
from sklearn import preprocessing
allFeature2_standardized = preprocessing.scale(allFeature2)

print(np.mean(allFeature2_standardized,axis=0))
print(np.std(allFeature2_standardized,axis=0))

[ 0.00000000e+00 -1.11022302e-18  0.00000000e+00]
[0. 1. 0.]


In [ ]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(allFeature1_standardized, only50Label, test_size=0.30, random_state=101,shuffle=True)

In [ ]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(allFeature2_standardized, only50Label, test_size=0.30, random_state=101,shuffle=True)

In [ ]:
from sklearn.svm import SVC

model = SVC(kernel='rbf',C=200)
model.fit(X_train1,y_train1)

SVC(C=200)

In [ ]:
from sklearn.svm import SVC

model = SVC(kernel='rbf',C=200)
model.fit(X_train2,y_train2)

SVC(C=200)

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
#prediction on test data
predictions1 = model.predict(X_test1)
print(predictions1)

#different evaluation matrices
print('accuracy_score==',accuracy_score(y_test1,predictions1))
print('confusion matrix==\n',confusion_matrix(y_test1,predictions1))
print(classification_report(y_test1,predictions1))

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
accuracy_score== 0.75
confusion matrix==
 [[45  0]
 [15  0]]
              precision    recall  f1-score   support

           0       0.75      1.00      0.86        45
           1       0.00      0.00      0.00        15

    accuracy                           0.75        60
   macro avg       0.38      0.50      0.43        60
weighted avg       0.56      0.75      0.64        60



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
#prediction on test data
predictions2 = model.predict(X_test2)
print(predictions2)

#different evaluation matrices
print('accuracy_score==',accuracy_score(y_test2,predictions2))
print('confusion matrix==\n',confusion_matrix(y_test2,predictions2))
print(classification_report(y_test2,predictions2))

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
accuracy_score== 0.75
confusion matrix==
 [[45  0]
 [15  0]]
              precision    recall  f1-score   support

           0       0.75      1.00      0.86        45
           1       0.00      0.00      0.00        15

    accuracy                           0.75        60
   macro avg       0.38      0.50      0.43        60
weighted avg       0.56      0.75      0.64        60



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
param_grid = {'C': [0.1,1, 10, 100, 1000], 'gamma': [1,0.1,0.01,0.001,0.0001], 'kernel': ['rbf']} ##for rbf kernel
# param_grid = {'C': [0.1,1, 10, 100, 1000], 'kernel': ['sigmoid']} ##for sigmoid kernel
# param_grid = {'C': [0.1,1, 10, 100, 1000], 'kernel': ['linear']} ##for linear kernel
# param_grid = {'C': [0.1,1, 10, 100, 1000], 'degree': [2,3,4,5,6], 'kernel': ['poly']} ##for polynomial kernel

### gamma=Kernel coefficient for ‘rbf’, ‘poly’ and ‘sigmoid’
### c=Regularization parameter

from sklearn.model_selection import GridSearchCV
grid1 = GridSearchCV(SVC(kernel='rbf'),param_grid,refit=True,verbose=2)
grid1.fit(X_train1,y_train1)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   0.0s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   0.0s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   0.0s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   0.0s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   0.0s
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time=   0.0s
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time=   0.0s
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time=   0.0s
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time=   0.0s
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time=   0.0s
[CV] END ......................C=0.1, gamma=0.01, kernel=rbf; total time=   0.0s
[CV] END ......................C=0.1, gamma=0.0

GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf']},
             verbose=2)

In [ ]:
param_grid = {'C': [0.1,1, 10, 100, 1000], 'gamma': [1,0.1,0.01,0.001,0.0001], 'kernel': ['rbf']} ##for rbf kernel
# param_grid = {'C': [0.1,1, 10, 100, 1000], 'kernel': ['sigmoid']} ##for sigmoid kernel
# param_grid = {'C': [0.1,1, 10, 100, 1000], 'kernel': ['linear']} ##for linear kernel
# param_grid = {'C': [0.1,1, 10, 100, 1000], 'degree': [2,3,4,5,6], 'kernel': ['poly']} ##for polynomial kernel

### gamma=Kernel coefficient for ‘rbf’, ‘poly’ and ‘sigmoid’
### c=Regularization parameter

from sklearn.model_selection import GridSearchCV
grid2 = GridSearchCV(SVC(kernel='rbf'),param_grid,refit=True,verbose=2)
grid2.fit(X_train2,y_train2)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   0.0s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   0.0s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   0.0s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   0.0s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   0.0s
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time=   0.0s
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time=   0.0s
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time=   0.0s
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time=   0.0s
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time=   0.0s
[CV] END ......................C=0.1, gamma=0.01, kernel=rbf; total time=   0.0s
[CV] END ......................C=0.1, gamma=0.0

GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf']},
             verbose=2)

In [ ]:
print(grid1.best_params_)
grid1.best_estimator_

{'C': 100, 'gamma': 0.1, 'kernel': 'rbf'}


SVC(C=100, gamma=0.1)

In [ ]:
grid_predictions1 = grid1.predict(X_test1)
print(confusion_matrix(y_test1,grid_predictions1))
print(accuracy_score(y_test1,grid_predictions1))

[[45  0]
 [15  0]]
0.75


In [ ]:
grid_predictions2 = grid2.predict(X_test2)
print(confusion_matrix(y_test2,grid_predictions2))
print(accuracy_score(y_test2,grid_predictions2))

[[45  0]
 [15  0]]
0.75


In [ ]:
model_linear = SVC(kernel='linear', C=1).fit(X_train1, y_train1)
model_rbf = SVC(kernel='rbf', gamma=1, C=1).fit(X_train1, y_train1)
model_poly = SVC(kernel='poly', degree=3, C=1).fit(X_train1, y_train1)
model_sig = SVC(kernel='sigmoid', C=1).fit(X_train1, y_train1)

In [ ]:
model_linear = SVC(kernel='linear', C=1).fit(X_train2, y_train2)
model_rbf = SVC(kernel='rbf', gamma=1, C=1).fit(X_train2, y_train2)
model_poly = SVC(kernel='poly', degree=3, C=1).fit(X_train2, y_train2)
model_sig = SVC(kernel='sigmoid', C=1).fit(X_train2, y_train2)


In [ ]:
linear_pred = model_linear.predict(X_test1)
poly_pred = model_poly.predict(X_test1)
rbf_pred = model_rbf.predict(X_test1)
sig_pred = model_sig.predict(X_test1)

# retrieve the accuracy and print it for all 4 kernel functions
accuracy_linear1 = model_linear.score(X_test1, y_test1)
accuracy_poly1 = model_poly.score(X_test1, y_test1)
accuracy_rbf1 = model_rbf.score(X_test1, y_test1)
accuracy_sig1 = model_sig.score(X_test1, y_test1)
print('Accuracy Linear Kernel:', accuracy_linear1)
print('Accuracy Polynomial Kernel:', accuracy_poly1)
print('Accuracy Radial Basis Kernel:', accuracy_rbf1)
print('Accuracy Sigmoid Kernel:', accuracy_sig1)
      
# creating a confusion matrix
cm_lin1 = confusion_matrix(y_test1, linear_pred)
cm_poly1 = confusion_matrix(y_test1, poly_pred)
cm_rbf1 = confusion_matrix(y_test1, rbf_pred)
cm_sig1 = confusion_matrix(y_test1, sig_pred)
print('cm_lin', '\n',cm_lin1, '\n')
print('cm_poly', '\n',cm_poly1 ,'\n')
print('cm_rbf','\n',cm_rbf1 ,'\n')
print('cm_sig','\n',cm_sig1 ,'\n')

Accuracy Linear Kernel: 0.75
Accuracy Polynomial Kernel: 0.75
Accuracy Radial Basis Kernel: 0.75
Accuracy Sigmoid Kernel: 0.75
cm_lin 
 [[45  0]
 [15  0]] 

cm_poly 
 [[45  0]
 [15  0]] 

cm_rbf 
 [[45  0]
 [15  0]] 

cm_sig 
 [[44  1]
 [14  1]] 



In [ ]:
linear_pred2 = model_linear.predict(X_test2)
poly_pred2 = model_poly.predict(X_test2)
rbf_pred2 = model_rbf.predict(X_test2)
sig_pred2 = model_sig.predict(X_test2)

# retrieve the accuracy and print it for all 4 kernel functions
accuracy_linear2 = model_linear.score(X_test2, y_test2)
accuracy_poly2 = model_poly.score(X_test2, y_test2)
accuracy_rbf2 = model_rbf.score(X_test2, y_test2)
accuracy_sig2 = model_sig.score(X_test2, y_test2)
print('Accuracy Linear Kernel:', accuracy_linear2)
print('Accuracy Polynomial Kernel:', accuracy_poly2)
print('Accuracy Radial Basis Kernel:', accuracy_rbf2)
print('Accuracy Sigmoid Kernel:', accuracy_sig2)
      
# creating a confusion matrix
cm_lin2 = confusion_matrix(y_test2, linear_pred2)
cm_poly2 = confusion_matrix(y_test2, poly_pred2)
cm_rbf2 = confusion_matrix(y_test2, rbf_pred2)
cm_sig2 = confusion_matrix(y_test2, sig_pred2)
print('cm_lin', '\n',cm_lin2, '\n')
print('cm_poly', '\n',cm_poly2 ,'\n')
print('cm_rbf','\n',cm_rbf2 ,'\n')
print('cm_sig','\n',cm_sig2 ,'\n')

Accuracy Linear Kernel: 0.75
Accuracy Polynomial Kernel: 0.75
Accuracy Radial Basis Kernel: 0.75
Accuracy Sigmoid Kernel: 0.7166666666666667
cm_lin 
 [[45  0]
 [15  0]] 

cm_poly 
 [[45  0]
 [15  0]] 

cm_rbf 
 [[45  0]
 [15  0]] 

cm_sig 
 [[42  3]
 [14  1]] 

